In [5]:
from fastquant import get_stock_data, backtest
from gaussian_hmm import *
import matplotlib.pyplot as plt

In [6]:
params = {
    'n_components': 2, 
    'algorithm': 'map', 
    'n_iter': 100, 
    'd': 5,
    'name':'GHMM'
}

ghmm = GHMM(params=params)

In [7]:
# back testing dates on AAPL 2019-01-01 to 2020-05-31
back_test_data = get_stock_data('AAPL', '2019-01-01', '2020-05-31')

[*********************100%***********************]  1 of 1 completed


In [8]:
# training data AAPL 2017-08-01 to 2019-01-01
training_data = get_stock_data('AAPL', '2017-08-01', '2019-01-01')

[*********************100%***********************]  1 of 1 completed


In [9]:
ghmm.train(train_data=training_data)

In [12]:
preds,actual = ghmm.predict(test_data=back_test_data)

Process ForkPoolWorker-1762:
Process ForkPoolWorker-1768:
Process ForkPoolWorker-1764:
Process ForkPoolWorker-1767:
Process ForkPoolWorker-1763:
Process ForkPoolWorker-1765:
Process ForkPoolWorker-1766:
Process ForkPoolWorker-1761:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python

KeyboardInterrupt: 